## Imports

In [ ]:
import pickle
import os
import sys
import numpy as np
import torch
import torchaudio
import matplotlib
import matplotlib.pyplot as plt
import argparse
import psutil
import GPUtil

# Define the base path where your pickle file is located
base_path2 = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/mask01-base-f128-t2-b24-lr1e-4-m400-pretrain_joint-asli'
base_path3 = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/mask01-base-f128-t2-b24-lr1e-4-m400-pretrain_joint-asli-20240412-172636'
base_path1_at_same_time = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/mask01-base-f128-t2-b24-lr1e-4-m400-pretrain_joint-asli-20240413-164417'
base_path_original = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/pretrained-base-f128-t2-b24-lr1e-4-m400-pretrain_joint-asli-original-20240416-103133'
base_path_shuffled = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/pretrained-base-f128-t2-b24-lr1e-4-m400-pretrain_joint-asli-shuffled-20240416-102831'
path_original_3 = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/pretrained-base-f128-t2-b24-lr1e-4-m400-pretrain_joint-asli-original-20240418-211014'
path_original_correctMean = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/pretrained-20240501-162648-original-base-f128-t2-b48-lr1e-4-m390-pretrain_joint-asli'

## Model Configuration

In [ ]:
# Construct the full path to the 'args.pkl' file
args_file_path2 = os.path.join(path_original_3, 'args.pkl')

# Load the arguments from the pickle file
with open(args_file_path2, 'rb') as f:
    args2 = pickle.load(f)

# Convert the Namespace to a dictionary if it is of that type
if isinstance(args2, argparse.Namespace):
    args_dict2 = vars(args2)
else:
    print("The loaded 'args' object is not an argparse.Namespace. Its type is:", type(args2))
    exit()

# Determine the maximum width of the argument names for alignment
max_key_length = max(len(key) for key in args_dict2.keys())

# Print the arguments in a structured table format
print(f"{'Argument':<{max_key_length}} | Value")
print("-" * (max_key_length + 3) + "+" + "-" * 30)  # Adjust 30 if you expect wider values

for key, value in args_dict2.items():
    print(f"{key:<{max_key_length}} | {value}")

## Epochs, Iterations and Time Required

### Compare the time required for training the model with 2 and 3 GPUs

In [ ]:
# open progress files
with open(os.path.join(base_path2, 'progress.pkl'), 'rb') as f:
    progress2 = pickle.load(f)

with open(os.path.join(base_path3, 'progress.pkl'), 'rb') as f:
    progress3 = pickle.load(f)

with open(os.path.join(base_path_original, 'progress.pkl'), 'rb') as f:
    progress_original = pickle.load(f)

with open(os.path.join(base_path_shuffled, 'progress.pkl'), 'rb') as f:
    progress_shuffled = pickle.load(f)

with open(os.path.join(path_original_3, 'progress.pkl'), 'rb') as f:
    progress_original_3 = pickle.load(f)

with open(os.path.join(path_original_correctMean, 'progress.pkl'), 'rb') as f:
    progress_original_correctMean = pickle.load(f)

In [ ]:
# get the iteration (is at the second position in the list of progress)
iteration2 = np.array([x[1] for x in progress2])
iteration3 = np.array([x[1] for x in progress3])
iteration_original = np.array([x[1] for x in progress_original])
iteration_shuffled = np.array([x[1] for x in progress_shuffled])
iter_original = np.array([x[1] for x in progress_original_3])
iter_original_correctMean = np.array([x[1] for x in progress_original_correctMean])*2

# get time (is at the fourth position in the list of progress)
time2 = np.array([x[3] for x in progress2])
time3 = np.array([x[3] for x in progress3])
time_original = np.array([x[3] for x in progress_original])
time_shuffled = np.array([x[3] for x in progress_shuffled])
time_orig = np.array([x[3] for x in progress_original_3])
time_original_correctMean = np.array([x[3] for x in progress_original_correctMean])

In [ ]:
# calculate total time required for all 800k iterations based on the number of iterations and the time per iteration
n_iterations = 800000
time_per_iteration2 = time2[10] / iteration2[10]
time_per_iteration3 = time3[10] / iteration3[10]
time_per_iteration_original = time_original[-1] / iteration_original[-1]
time_per_iteration_shuffled = time_shuffled[-1] / iteration_shuffled[-1]
time_per_iter_orig = time_orig[-1] / iter_original[-1]
time_per_iter_orig_correctMean = time_original_correctMean[-1] / iter_original_correctMean[-1]

total_time2 = time_per_iteration2 * n_iterations
total_time3 = time_per_iteration3 * n_iterations
total_time_original = time_per_iteration_original * n_iterations
total_time_shuffled = time_per_iteration_shuffled * n_iterations
total_time_orig = time_per_iter_orig * n_iterations
total_time_orig_correctMean = time_per_iter_orig_correctMean * n_iterations/2

print(f"Total time for 2 GPUs: {total_time2/3600:.2f} hours")
print(f"Total time for 3 GPUs: {total_time3/3600:.2f} hours")
print(f"Total time for original: {total_time_original/3600:.2f} hours")
print(f"Total time for shuffled: {total_time_shuffled/3600:.2f} hours")
print(f"Total time for original 3 PGUs: {total_time_orig/3600:.2f} hours")
print(f"Total time for original correctMean: {total_time_orig_correctMean/3600:.2f} hours")

In [ ]:
# plot iter vs time all in one plot
plt.plot(iteration2, time2, label='2 GPUs')
plt.plot(iteration3, time3, label='3 GPUs')
plt.plot(iteration_original, time_original, label='original')
plt.plot(iteration_shuffled, time_shuffled, label='shuffled')
plt.plot(iter_original, time_orig, label='original 3 GPUs')
plt.xlabel('Iteration')
plt.ylabel('Time [s]')
plt.legend()
plt.show()


## Training and Evaluation Loss

In [ ]:
# Construct the full path to the 'result.csv' file
result_file_path3 = os.path.join(base_path3, 'result.csv')

# Load the result from the csv file
result3 = np.genfromtxt(result_file_path3, delimiter=',')

# Extract the columns from the result
acc_train3 = result3[:, 0] # The first column
loss_train3 = result3[:, 1] # The second column
acc_eval3 = result3[:, 2] # The third column
mse_eval3 = result3[:, 3] # The fourth column

### Compare the training and evaluation loss for the model trained with original and shuffled spectrograms

In [ ]:
# Construct the full path to the 'result.csv' file
result_file_path_original = os.path.join(base_path_original, 'result.csv')
result_file_path_shuffled = os.path.join(base_path_shuffled, 'result.csv')

# Load the result from the csv file
result_original = np.genfromtxt(result_file_path_original, delimiter=',')
result_shuffled = np.genfromtxt(result_file_path_shuffled, delimiter=',')
print("shape of result_original: ", result_original.shape)
print("shape of result_shuffled: ", result_shuffled.shape)

# Extract the columns from the result

acc_train_original = result_original[:, 0] # The first column
loss1_train_original = result_original[:, 1] # The second column
loss2_train_original = result_original[:, 2] # The third column
acc_eval_original = result_original[:, 3] # The fourth column
loss1_eval_original = result_original[:, 4] # The fifth column
loss2_eval_original = result_original[:, 5] # The sixth column

acc_train_shuffled = result_shuffled[:, 0] # The first column
loss1_train_shuffled = result_shuffled[:, 1] # The second column
loss2_train_shuffled = result_shuffled[:, 2] # The third column
acc_eval_shuffled = result_shuffled[:, 3] # The fourth column
loss1_eval_shuffled = result_shuffled[:, 4] # The fifth column
loss2_eval_shuffled = result_shuffled[:, 5] # The sixth column

learning_rate_original = result_original[:, 6] # The seventh column
learning_rate_shuffled = result_shuffled[:, 6] # The seventh column


# Define the format for each column
header_format = " {:>5}  | {:<10} | {:<10} | {:<10} | {:<10} | {:<10} | {:<10}"
row_format = "{:>5}k  | {:<10.5f} | {:<10.5f} | {:<10.5f} | {:<10.5f} | {:<10.5f} | {:<10.5f}"

In [ ]:
# Orignal
print("Original Spectrograms:")
print("-" * 86)
print(header_format.format("iter", "acc_train", "loss1_tr", "loss2_tr", "acc_ev", "loss1_ev", "loss2_ev"))
print("-" * 86)  # Adjust the total length to fit your headers and column data
for i in range(len(acc_train_original)):
    print(row_format.format(iteration_original[i]/1e3, acc_train_original[i], loss1_train_original[i], loss2_train_original[i], acc_eval_original[i], loss1_eval_original[i], loss2_eval_original[i]))
print("-" * 86 + "\n")

In [ ]:
# Shuffled
print("Shuffled Spectrograms:")
print("-" * 86)
print(header_format.format("iter", "acc_train", "loss1_tr", "loss2_tr", "acc_ev", "loss1_ev", "loss2_ev"))
print("-" * 86)  # Adjust the total length to fit your headers and column data
for i in range(len(acc_train_shuffled)):
    print(row_format.format(iteration_shuffled[i]/1e3, acc_train_shuffled[i], loss1_train_shuffled[i], loss2_train_shuffled[i], acc_eval_shuffled[i], loss1_eval_shuffled[i], loss2_eval_shuffled[i]))
print("-" * 86 + "\n")

In [ ]:
result_gong_path = '/home/bosfab01/SpeakerVerificationBA/pretraining/result_gong.csv'
result_original_path = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/pretrained-base-f128-t2-b24-lr1e-4-m400-pretrain_joint-asli-original-20240416-103133/result.csv'
result_shuffled_path = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/pretrained-base-f128-t2-b24-lr1e-4-m400-pretrain_joint-asli-shuffled-20240416-102831/result.csv'
result_3GPU_path = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/mask01-base-f128-t2-b24-lr1e-4-m400-pretrain_joint-asli-20240412-172636/result.csv'
result_original_3GPU_path = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/pretrained-base-f128-t2-b24-lr1e-4-m400-pretrain_joint-asli-original-20240418-211014/result.csv'
result_original_correctMean_path = '/home/bosfab01/SpeakerVerificationBA/pretraining/exp/pretrained-20240501-162648-original-base-f128-t2-b48-lr1e-4-m390-pretrain_joint-asli/result.csv'

result_gong = np.genfromtxt(result_gong_path, delimiter=',')
result_original = np.genfromtxt(result_original_path, delimiter=',')
result_shuffled = np.genfromtxt(result_shuffled_path, delimiter=',')
result_3GPU = np.genfromtxt(result_3GPU_path, delimiter=',')
result_original_3GPU = np.genfromtxt(result_original_3GPU_path, delimiter=',')
result_original_correctMean = np.genfromtxt(result_original_correctMean_path, delimiter=',')

# print shapes of the arrays
print("shape of result_gong: ", result_gong.shape)
print("shape of result_original: ", result_original.shape)
print("shape of result_shuffled: ", result_shuffled.shape)
print("shape of result_3GPU: ", result_3GPU.shape)
print("shape of result_original_3GPU: ", result_original_3GPU.shape)
print("shape of result_original_correctMean: ", result_original_correctMean.shape)

In [ ]:
# function to return columns of a numpy array
def get_column(array):
    for i in range(array.shape[1]):
        yield array[:, i]

acc_tr_gong, loss_tr_gong, acc_ev_gong, mse_ev_gong, lr_gong = get_column(result_gong)
acc_tr_3GPU, loss_tr_3GPU, acc_ev_3GPU, mse_ev_3GPU, _ = get_column(result_3GPU)
acc_tr_original, loss1_tr_original, loss2_tr_original, acc_ev_original, loss1_ev_original, loss2_ev_original, _ = get_column(result_original)
acc_tr_shuffled, loss1_tr_shuffled, loss2_tr_shuffled, acc_ev_shuffled, loss1_ev_shuffled, loss2_ev_shuffled, _ = get_column(result_shuffled)
acc_tr_original_3GPU, loss1_tr_original_3GPU, loss2_tr_original_3GPU, acc_ev_original_3GPU, loss1_ev_original_3GPU, loss2_ev_original_3GPU, lr_original_3GPU = get_column(result_original_3GPU)
acc_tr_original_correctMean, loss1_tr_original_correctMean, loss2_tr_original_correctMean, acc_ev_original_correctMean, loss1_ev_original_correctMean, loss2_ev_original_correctMean, lr_original_correctMean = get_column(result_original_correctMean)

iteration_gong = np.arange(1, len(acc_tr_gong)+1) * 4000

In [ ]:
# plot all the stats
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(14, 10))

axs[0, 0].plot(iteration_gong[:len(acc_tr_original_3GPU)+5]/1e3, acc_tr_gong[:len(acc_tr_original_3GPU)+5], label='gong')
axs[0, 0].plot(iteration3[:len(acc_tr_original_3GPU)+5]/1e3, acc_tr_3GPU[:len(acc_tr_original_3GPU)+5], label='original 1024')
axs[0, 0].plot(iter_original_correctMean/1e3, acc_tr_original_correctMean, label='original correctMean')
# axs[0, 0].plot(iteration_original/1e3, acc_tr_original, label='original')
# axs[0, 0].plot(iteration_shuffled/1e3, acc_tr_shuffled, label='shuffled')
axs[0, 0].plot(iter_original/1e3, acc_tr_original_3GPU, label='original 998')
#axs[0, 0].set_title('Accuracy Training')
#axs[0, 0].set_xticks(np.arange(0, len(acc_tr_original)+5, 1))
axs[0, 0].set_xlabel('Iteration [k]')
axs[0, 0].set_ylabel('Accuracy Training')
axs[0, 0].legend()
axs[0, 0].grid()


axs[0, 1].plot(iteration_gong[:len(acc_tr_original_3GPU)+5]/1e3, loss_tr_gong[:len(acc_tr_original_3GPU)+5], label='gong')
axs[0, 1].plot(iteration3[:len(acc_tr_original_3GPU)+5]/1e3, loss_tr_3GPU[:len(acc_tr_original_3GPU)+5], label='original 1024')
axs[0, 1].plot(iter_original_correctMean/1e3, loss1_tr_original_correctMean+10*loss2_tr_original_correctMean, label='original correctMean')
# axs[0, 1].plot(iteration_original/1e3, loss1_tr_original+10*loss2_tr_original, label='original')
# axs[0, 1].plot(iteration_shuffled/1e3, loss1_tr_shuffled+10*loss2_tr_shuffled, label='shuffled')
axs[0, 1].plot(iter_original/1e3, loss1_tr_original_3GPU+10*loss2_tr_original_3GPU, label='original 998')
#axs[0, 1].set_title('Loss Training')
#axs[0, 1].set_xticks(np.arange(0, len(acc_tr_original)+5, 1))
axs[0, 1].set_xlabel('Iteration [k]')
axs[0, 1].set_ylabel('Loss Training')
axs[0, 1].legend()
axs[0, 1].grid()

axs[1, 0].plot(iteration_gong[:len(acc_tr_original_3GPU)+5]/1e3, acc_ev_gong[:len(acc_tr_original_3GPU)+5], label='gong')
axs[1, 0].plot(iteration3[:len(acc_tr_original_3GPU)+5]/1e3, acc_ev_3GPU[:len(acc_tr_original_3GPU)+5], label='original 1024')
axs[1, 0].plot(iter_original_correctMean/1e3, acc_ev_original_correctMean, label='original correctMean')
# axs[1, 0].plot(iteration_original/1e3, acc_ev_original, label='original')
# axs[1, 0].plot(iteration_shuffled/1e3, acc_ev_shuffled, label='shuffled')
axs[1, 0].plot(iter_original/1e3, acc_ev_original_3GPU, label='original 998')
#axs[1, 0].set_title('Accuracy Evaluation')
#axs[1, 0].set_xticks(np.arange(0, len(acc_tr_original)+5, 1))
axs[1, 0].set_xlabel('Iteration [k]')
axs[1, 0].set_ylabel('Accuracy Evaluation')
axs[1, 0].legend()
axs[1, 0].grid()

axs[1, 1].plot(iteration_gong[:len(acc_tr_original_3GPU)+5]/1e3, mse_ev_gong[:len(acc_tr_original_3GPU)+5], label='gong')
axs[1, 1].plot(iteration3[:len(acc_tr_original_3GPU)+5]/1e3, mse_ev_3GPU[:len(acc_tr_original_3GPU)+5], label='original 1024')
axs[1, 1].plot(iter_original_correctMean/1e3, loss2_ev_original_correctMean, label='original correctMean')
# axs[1, 1].plot(iteration_original/1e3, loss2_ev_original, label='original')
# axs[1, 1].plot(iteration_shuffled/1e3, loss2_ev_shuffled, label='shuffled')
axs[1, 1].plot(iter_original/1e3, loss2_ev_original_3GPU, label='original 998')
#axs[1, 1].set_title('MSE Evaluation')
#axs[1, 1].set_xticks(np.arange(0, len(acc_tr_original)+5, 1))
axs[1, 1].set_xlabel('Iteration [k]')
axs[1, 1].set_ylabel('MSE Evaluation')
axs[1, 1].legend()
axs[1, 1].grid()

plt.show()

In [ ]:
# plot learning rate on log scale
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(iteration_original/1e3, learning_rate_original, label='original')
ax.plot(iteration_shuffled/1e3, learning_rate_shuffled, label='shuffled')
ax.plot(iter_original/1e3, lr_original_3GPU, label='original 998')
ax.plot(iteration_gong/1e3, lr_gong, label='gong')
ax.set_yscale('log')
ax.set_xlabel('Iteration [k]')
ax.set_ylabel('Learning Rate')
ax.legend()
ax.grid()
plt.show()

In [ ]:
#plt.plot(iteration_shuffled/1e3, loss2_ev_shuffled, label='shuffled')
plt.plot(iteration_shuffled/1e3, loss1_ev_shuffled, label='shuffled')

## testing the dataloader

In [ ]:
# Navigate up one level to the 'pretraining' directory, where 'dataloader.py' is located
sys.path.append(os.path.abspath('../'))

import dataloader

### demonstration of mismatch between target length and input length

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataloader.AudioDataset(
        dataset_json_file='/home/bosfab01/SpeakerVerificationBA/data/audioset2M_librispeech960.json',
        audio_conf={
            'num_mel_bins': 128,
            'target_length': 1024,
            'freqm': 0,
            'timem': 0,
            'mixup': 0,
            'dataset': 'asli',
            'mean': -3.6925695,
            'std': 4.020388,
            'noise': False,
            'mode': 'train',
            'shuffle_frames': False
        },
        label_csv='/home/bosfab01/SpeakerVerificationBA/data/label_information.csv'
    ),
    batch_size=24,
    shuffle=True,
    num_workers=16,
    pin_memory=True,
    drop_last=True
)

### get one batch of data from the dataloader and display the images

In [ ]:
# Create an iterator from the DataLoader
data_iterator = iter(train_loader)

# Fetch the first batch
audio_input, labels = next(data_iterator)

# Print out the details to see what the batch contains
print("Audio input shape:", audio_input.shape)
print("Labels shape:", labels.shape)

In [ ]:
# print the last 10 spectra of the first sample in the batch
# this is to check if the number of frames matches the target_length

colors = plt.cm.viridis(np.linspace(0, 1, 10))  # Generate 10 colors from the 'viridis' colormap

for i in range(10, 0, -1):
    markerline, stemlines, baseline = plt.stem(audio_input[0, -i, :], linefmt='-', basefmt=" ")
    plt.setp(stemlines, 'linewidth', 2, 'color', colors[10-i])  # Set the color and line width
    plt.setp(markerline, 'marker', '')  # No marker at the end

plt.legend([f'Spectrum {-i}' for i in range(10, 0, -1)])
plt.xlabel('Frequency Bins')
plt.ylabel('Magnitude')
plt.title('Spectra of the Last 10 Frames')
plt.grid(True)
plt.show()

# Print the last spectrum of the first sample
print(audio_input[0, -1, :])

In [ ]:
# Adjusted function to plot a spectrogram with the correct orientation
def plot_spectrogram(spectrogram, ax, title="Spectrogram"):
    # Transpose the spectrogram to align the axes correctly
    ax.imshow(spectrogram.T.cpu().numpy(), aspect='auto', origin='lower', cmap='viridis')
    ax.set_title(title)
    ax.set_xlim(0, spectrogram.shape[0])
    ax.set_xlabel('Time Frames')
    ax.set_ylabel('Mel Frequency Bins')

for i in range(3):
    fig, ax = plt.subplots(figsize=(10, 1.5))
    plot_spectrogram(audio_input[i, :, :], ax, title=f'Spectrogram of sample {i}')
    plt.show()